In [1]:
!pip install pandas numpy matplotlib tiktoken openai

In [2]:
import pandas as pd
import numpy as np
import matplotlib

In [3]:
contents = []

for i in range(1, 8):
    file_name = f"./control_points/{i}.txt"
    try:
        with open(file_name, 'r', encoding="utf-8") as file:
            content = file.read()
            contents.append({'content': content})
    except FileNotFoundError:
        print(f"File {file_name} not found.")

df = pd.DataFrame(contents, columns=['content'])

In [5]:
import re

In [6]:
pd.options.mode.chained_assignment = None 
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['content']= df["content"].apply(lambda x : normalize_text(x))

In [7]:
import tiktoken

In [8]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["content"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]

In [9]:
import os
from openai import AzureOpenAI

In [10]:
os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2023-05-15",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['embedding'] = df["content"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [11]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(_df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    _df["similarities"] = _df["embedding"].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        _df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

In [12]:
res = search_docs(df, "Le token de l'utilisateur se retrouve régulièrement invalidé dans un délai de moins de 180 jours. "
, top_n=4)

,content,n_tokens,embedding,similarities
6,Point de contrôle 13 ======= # Catégorie : Aut...,323,"[-0.00961134023964405, -0.00813981145620346, 0...",0.876890
0,Point de contrôle 1 ======= # Catégorie : Tier...,143,"[-0.017865363508462906, -0.019515523687005043,...",0.766913
5,Point de contrôle 23 ======= # Catégorie : Don...,386,"[-0.009391171857714653, 0.007496757432818413, ...",0.766757
3,Point de contrôle 4 ======= # Catégorie : Tier...,513,"[-0.027942441403865814, -0.019337281584739685,...",0.761505


In [51]:
system_prompt = """Objectif Principal : Tu es un assistant qui doit permettre aux utilisateurs de l'ACPR d'accéder facilement aux documents normatifs relatifs à la supervision des activités des établissements financiers et de vérifier la conformité des phrases relatives aux produits financiers aux règlements en vigueur.
1. Recherche Documentaire :
- Si l’utilisateur te demande de rechercher des documents normatifs en utilisant des mots-clés, des catégories spécifiques ou des références règlementaires, tu dois uniquement citer les documents faisant référence à ces éléments dans une liste ordonnée.
- Si la formulation de l’utilisateur est trop complexe, demande-lui segmenter ses requêtes
- Si l’utilisateur ne parvient pas à trouver ce qu’il cherche, donne-lui des exemples de phrases à renseigner. Par exemple, donne-lui des mots clés ou des catégories. 
- Si tu lis une abréviation dans un document ou si tu lis une abréviation dans la phrase de l’utilisateur et que tu ne comprends pas l’abréviation, cite l’abréviation et demande à l’utilisateur de la définir. 
2. Accès aux Informations :
- Tu dois extraire de manière précise les informations pertinentes des documents normatifs en réponse aux requêtes de l'utilisateur.
- Tu dois restituer le contexte entourant une information pour une meilleure compréhension. Fais bien attention, à séparer la citation du texte normatif des informations contextuelles. Le texte normatif doit être clairement identifiable et ne doit pas avoir été modifié. 
3. Interaction Naturelle :
- Favorise une interaction conversationnelle naturelle avec l'utilisateur en comprenant le langage courant et en fournissant des réponses compréhensibles.
- Réponds avec un langage formel est clair. Ce que tu écris doit pouvoir être présenté dans des rapports officiels. 
4. Conformité des Phrases aux Règlements :
- Si l’utilisateur demande la conformité d’une phrase produis une réponse dans un fichier JSON. Le fichier JSON doit être structuré ainsi :
{
"Catégorie" : "émission de dette",
"Numéro du point de contrôle" : "1",
"Intitulé du point de contrôle" : "Les dettes doivent être émises publiquement et être disponible à tout le monde",
"Base légale" : [
"1. Article 1", 
"2. Article 2 du Code monétaire"],
"Requête" : "<insère la phrase que l’utilisateur a rentrée>",
"Conformité" : "oui / non / je ne sais pas",
"Procédure à suivre" : "En cas de non-conformité, il faut informer l’établissement de la non-conformité et demander les éléments de remédiation dans un délai de 3 mois / Aucune procédure à suivre",
"Résumé": <insére ta réponse en langage naturel en te référant à l'instruction "3. Interaction Naturelle">
}
Si la phrase est jugée conforme, écris « Aucune procédure à suivre » dans le dossier JSON, sinon cite la préocédure à suivre. 
- Si la phrase n’est pas en relation avec le point de contrôle, produis un fichier JSON expliquant que la phrase n’est pas en relation avec le point de contrôle. 
- Si des champs du fichier JSON ne sont pas applicables au contexte, ne les inclus pas.
5. Gestion des Erreurs et Ambiguïtés :
- Tu dois gérer les situations où une requête est ambiguë ou incomplète en demandant des clarifications.
- Si l’utilisateur fait une faute de frappe, corrige-la en citant la correction et réponds à la question posée par la phrase corrigée. 
- Si tu ne connais pas la réponse, dis-le et ne cherche pas à rajouter d’autres éléments à part des questions pour plus de précision.
6. Mises à Jour Légales :
- Tu dois informer les utilisateurs des modifications récentes dans la législation financière en précisant les dates. 
7. Assistance et Support :
- Fournis un support contextuel pour aider les utilisateurs à formuler des requêtes de manière efficace. Attention, cela ne doit jamais modifier les textes normatifs dans tes réponses.
- Propose des questions annexes une fois que tu as satisfait la requête de l’utilisateur.
8. Point de contrôle :
- Voici des points de contrôle qui te permettent de vérifier la conformité des phrases et ancrer tes réponses : 

"""

In [39]:
os.environ["AZURE_OPENAI_DeploymentId"] = "gpt-4-turbo"

def generate_answer(prompt, system_prompt, trace = []):
  if (len(trace) == 0):
    trace = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": prompt},
    ]
  else:
    trace.append({"role": "user", "content": prompt})
  res = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DeploymentId"),
    response_format={"type": "json_object"},
    messages=trace
  ).choices[0].message
  trace.append({
    "role": res.role,
    "content": res.content
  })
  
  return {
    "content": res.content,
    "trace": trace
  }

def process_prompt(prompt, trace = []):
  res = search_docs(df, prompt, top_n=2, to_print=False)
  sys_prompt = system_prompt + "\n".join(res["content"])
  answer = generate_answer(prompt, sys_prompt, trace)
  return ({"answer": answer["content"], "docs": res, "trace": answer["trace"]})

In [15]:
df

,content,n_tokens,embedding,similarities
0,Point de contrôle 1 ======= # Catégorie : Tier...,143,"[-0.017865363508462906, -0.019515523687005043,...",0.766913
1,Point de contrôle 2 ======= # Catégorie : Tier...,946,"[-0.02869136817753315, -0.020042575895786285, ...",0.729033
2,Point de contrôle 3 ======= # Catégorie : Tier...,236,"[-0.009903984144330025, -0.022082369774580002,...",0.755501
3,Point de contrôle 4 ======= # Catégorie : Tier...,513,"[-0.027942441403865814, -0.019337281584739685,...",0.761505
4,Point de contrôle 5 ======= # Catégorie : Tier...,777,"[-0.02356548421084881, -0.022029506042599678, ...",0.748147
5,Point de contrôle 23 ======= # Catégorie : Don...,386,"[-0.009391171857714653, 0.007496757432818413, ...",0.766757
6,Point de contrôle 13 ======= # Catégorie : Aut...,323,"[-0.00961134023964405, -0.00813981145620346, 0...",0.876890


In [48]:
res = process_prompt("Donne-moi le point de contrôle sur l'authentification")
res["answer"]

'\n    {\n    "Catégorie" : "Authentification à l’interface d’accès",\n    "Numéro du point de contrôle" : "13",\n    "Intitulé du point de contrôle" : "L’interface d’accès doit prévoir une exemption d’authentification forte de 180 jours pour les prestataires de services d’information sur les comptes. Concrètement, les tokens doivent être valides au moins pour une durée de 180 jours. Dans l’attente, il ne peut être fait de différences en termes de fréquence d’authentification entre les interfaces d’accès et l’accès direct aux comptes de paiement mis à disposition par l’établissement teneur de compte à ses clients.",\n    "Base légale" : [\n    "1. Article 10 du Règlement délégué n°2018/389 – RTS SCA-CSC",\n    "2. Point 30 de l’Opinion EBA/OP/2020/10"\n    ],\n    "Exemple de conformité" : "La durée de validité du token pour accéder aux comptes de paiements est de 190 jours.",\n    "Exemple de non-conformité" : "La durée de vie du refresh token est toujours de 90 jours au lieu des 180 

In [45]:
res["answer"]

'{\n  "Catégorie" : "Données communiquées par les Prestataires de services de paiement gestionnaire de compte (PSPGC) aux Tiers prestataires de service de paiements (TPP)",\n  "Numéro du point de contrôle" : "23",\n  "Intitulé du point de contrôle" : "Toutes les informations liées aux comptes de paiement, mises à la disposition de l’utilisateur de services de paiement doivent être disponibles dans l’API (par exemple et sans exhaustivité : les libellés des comptes, la distinction des cartes à débit différé, la date des opérations, les opérations en attente..) selon le principe d’une non- discrimination entre banque en ligne et interface dédiée. Les services à valeur ajoutée fournis par l’établissement teneur de compte ne sont pas concernés par cette disposition (exemple : aide à la gestion du budget, service Paylib..).",\n  "Base légale" : [\n    "1. Article 67 2. d) et 3. b) de la DSP2.",\n    "2. Article L133-41 du Code monétaire et financier.",\n    "3. Article 36 (1) (a) du Règlemen

In [49]:
res = process_prompt("Donne moi la base légale", trace=res["trace"])

In [50]:
res["answer"]

'\n    {\n    "Base légale" : [\n    "1. Article 10 du Règlement délégué n°2018/389 – RTS SCA-CSC",\n    "2. Point 30 de l’Opinion EBA/OP/2020/10"\n    ]\n    }'

In [68]:
import pandas as pd
import numpy as np
import json
import os
from openai import AzureOpenAI

class VegaAssistant:
  def __init__(self, embeddings_source, system_prompt):
    os.environ["AZURE_OPENAI_KEY"] = "7e93421f46cd4680831023addcb0f42d"
    os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
    os.environ["AZURE_OPENAI_DeploymentId"] = "gpt-4-turbo"
    self.client = AzureOpenAI(
      api_key = os.getenv("AZURE_OPENAI_KEY"),  
      api_version = "2023-05-15",
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    contents = []
    obj = json.load(open(embeddings_source, "r"))
    for i in range(len(obj)):
      contents.append({'content': obj[i]["content"], "embedding": obj[i]["embedding"]})
    
    self.embeddings = pd.DataFrame(contents, columns=['content', 'embedding'])
    self.system_prompt = system_prompt
    self.trace = []

  def _cosine_similarity(self, a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
  
  def _get_embedding(self, text, model="ada-002"): # model = "deployment_name"
    return self.client.embeddings.create(input = [text], model=model).data[0].embedding

  def search_docs(self, _embeddings: pd.DataFrame, user_query, top_n=4):
      embedding = self._get_embedding(
          user_query,
          model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
      )
      embeddings = _embeddings.copy()
      embeddings["similarities"] = embeddings["embedding"].apply(lambda x: self._cosine_similarity(x, embedding))

      res = (
          embeddings.sort_values("similarities", ascending=False)
          .head(top_n)
      )
      return res

  def generate_answer(self, prompt, system_prompt):
    if (len(self.trace) == 0):
      self.trace = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
      ]
    else:
      self.trace.append({"role": "user", "content": prompt})
    res = self.client.chat.completions.create(
      model=os.getenv("AZURE_OPENAI_DeploymentId"),
      response_format={"type": "json_object"},
      messages=self.trace
    ).choices[0].message
    self.trace.append({
      "role": res.role,
      "content": res.content
    })
    
    return res.content

  def chat(self, prompt):
    res = self.search_docs(self.embeddings, prompt, top_n=2)
    sys_prompt = self.system_prompt + "\n".join(res["content"])
    answer = self.generate_answer(prompt, sys_prompt)
    return ({"answer": answer, "docs": res })

In [69]:
assistant = VegaAssistant("control_points_with_embeddings.json", system_prompt)

res = assistant.chat("Donne-moi le point de contrôle sur l'authentification")

In [70]:
assistant.trace

[{'role': 'system',
  'content': 'Objectif Principal : Tu es un assistant qui doit permettre aux utilisateurs de l\'ACPR d\'accéder facilement aux documents normatifs relatifs à la supervision des activités des établissements financiers et de vérifier la conformité des phrases relatives aux produits financiers aux règlements en vigueur.\n1. Recherche Documentaire :\n- Si l’utilisateur te demande de rechercher des documents normatifs en utilisant des mots-clés, des catégories spécifiques ou des références règlementaires, tu dois uniquement citer les documents faisant référence à ces éléments dans une liste ordonnée.\n- Si la formulation de l’utilisateur est trop complexe, demande-lui segmenter ses requêtes\n- Si l’utilisateur ne parvient pas à trouver ce qu’il cherche, donne-lui des exemples de phrases à renseigner. Par exemple, donne-lui des mots clés ou des catégories. \n- Si tu lis une abréviation dans un document ou si tu lis une abréviation dans la phrase de l’utilisateur et que tu

In [71]:
res = assistant.chat("Donne moi la base légale")

In [72]:
assistant.trace

[{'role': 'system',
  'content': 'Objectif Principal : Tu es un assistant qui doit permettre aux utilisateurs de l\'ACPR d\'accéder facilement aux documents normatifs relatifs à la supervision des activités des établissements financiers et de vérifier la conformité des phrases relatives aux produits financiers aux règlements en vigueur.\n1. Recherche Documentaire :\n- Si l’utilisateur te demande de rechercher des documents normatifs en utilisant des mots-clés, des catégories spécifiques ou des références règlementaires, tu dois uniquement citer les documents faisant référence à ces éléments dans une liste ordonnée.\n- Si la formulation de l’utilisateur est trop complexe, demande-lui segmenter ses requêtes\n- Si l’utilisateur ne parvient pas à trouver ce qu’il cherche, donne-lui des exemples de phrases à renseigner. Par exemple, donne-lui des mots clés ou des catégories. \n- Si tu lis une abréviation dans un document ou si tu lis une abréviation dans la phrase de l’utilisateur et que tu